In [1]:
import pandas as pd
import time

import requests
from bs4 import BeautifulSoup

In [2]:
from tqdm.notebook import tqdm

In [3]:
# change full text df
df = pd.read_csv("../files/df_key.csv")

In [5]:
# Error가 아닌 case
df[df['meta_keywords'] != 'Error']

,titles,links,year,meta_keywords


In [30]:
def crawling_keywords(url):
    # url Html 구조 긁어오기
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    try:
        div_abstract = soup.find('div', class_='abstract')   
        div_abstract.find('div', class_='abstract-content selected').decompose()

        keywords = div_abstract.find('p').get_text(strip=True)
        keywords = keywords.split(':')[1]
        meta_keywords = keywords = keywords.split('; ')
        meta_keywords = ','.join(meta_keywords)
        meta_keywords = meta_keywords[:-1]
    except:
        meta_keywords = 'Error'

    return meta_keywords


In [31]:
crawling_keywords('https://pubmed.ncbi.nlm.nih.gov/33246246/')

'16S rRNA,Altitude,Diversity,Oral microbiome.'

In [42]:
url = 'https://pubmed.ncbi.nlm.nih.gov/33246246/'

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')


In [43]:
# line by line 
div_abstract = soup.find('div', class_='abstract')
div_abstract


<div class="abstract" id="abstract">
<h2 class="title">
        Abstract
        
      </h2>
<div class="abstract-content selected" id="enc-abstract">
<p>
<strong class="sub-title">
          Objectives:
        </strong>
      
      The aim of this study was to obtain greater insight into the environmental and genetic factors affecting the oral microbiome.
    </p>
<p>
<strong class="sub-title">
          Design:
        </strong>
      
      To this end, we investigated the oral microbiome composition in Han and Zang populations living at different altitudes. The saliva microbiome in 115 individuals from Zang and Han populations living at different altitudes was analyzed using the 16 s rRNA gene sequencing method on the Illumina MiSeq platform. The dominant species in the oral microbiome were verified by quantitative real-time polymerase chain reaction (qPCR) analysis.
    </p>
<p>
<strong class="sub-title">
          Results:
        </strong>
      
      The Han population, liv

In [44]:
div_abstract.find('div', class_='abstract-content selected').decompose()

In [45]:
div_abstract

<div class="abstract" id="abstract">
<h2 class="title">
        Abstract
        
      </h2>

<p>
<strong class="sub-title">
          Keywords:
        </strong>
      
      16S rRNA; Altitude; Diversity; Oral microbiome.
    </p>
</div>

In [59]:
keywords = div_abstract.find('p').get_text(strip=True)
keywords

'Keywords:16S rRNA; Altitude; Diversity; Oral microbiome.'

In [61]:
keywords.split(':')

['Keywords', '16S rRNA; Altitude; Diversity; Oral microbiome.']

In [63]:
keywords = keywords.split(':')[1]
keywords

'16S rRNA; Altitude; Diversity; Oral microbiome.'

In [67]:
# split to list
meta_keywords = keywords.split('; ')
meta_keywords

['16S rRNA', 'Altitude', 'Diversity', 'Oral microbiome.']

In [69]:
# list to string by join
meta_keywords = ','.join(meta_keywords)
meta_keywords

'16S rRNA,Altitude,Diversity,Oral microbiome.'

In [94]:
import time


for i, url in enumerate(tqdm(df.links)):
    keywords = crawling_keywords(url)
    df['meta_keywords'][i] = keywords
    
    if i % 1000 == 0:
        time.sleep(10)
    

  0%|          | 0/104319 [00:00<?, ?it/s]

/Users/geoffrey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KeyboardInterrupt: 

In [106]:
# year over 2010 paper crawling

df.head()

test_df = df[df['year'] >= 2015]

In [111]:
from copy import copy

In [112]:
for i, url in enumerate(tqdm(test_df.links)):
    keywords = crawling_keywords(url)
    test_df['meta_keywords'][i] = copy(keywords)
    
    if i % 1000 == 0:
        time.sleep(10)
    

  0%|          | 0/1812 [00:00<?, ?it/s]

/Users/geoffrey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [89]:
fruit_list = ['apple', 'banana', 'pineapple', 'orange', 'grape']

for fruit in fruit_list:
    print(fruit)
    
print("=====================")

for i in range(len(fruit_list)):
    print(i, fruit_list[i])
    
print("=====================")

for i, fruit in enumerate(fruit_list):
    print(i, fruit)

apple
banana
pineapple
orange
grape
0 apple
1 banana
2 pineapple
3 orange
4 grape
0 apple
1 banana
2 pineapple
3 orange
4 grape


In [127]:
term_frequency = df[df.meta_keywords != 'Error']
term_frequency = list(term_frequency['meta_keywords'].values)

In [141]:
# list map lambda example
# 모든 element들에 대해서 똑같은 작업을 하고싶을때 사용
test_list = ['apple', 'banana', 'pineapple', 'grape']


# 모든 element들의 앞 2글자만 따오기
# map(함수, 리스트)



lambda_tmp = list(map(lambda x: x[:2], test_list))
print("람다 테스트")
print(lambda_tmp)


# 같은 표현식
def get_two_string(x):
    return x[:2]

function_tmp = list(map(get_two_string, test_list))
function_tmp = list(map(lambda x: get_two_string(x), test_list))

print('함수 테스트')
print(function_tmp)

람다 테스트
['ap', 'ba', 'pi', 'gr']
함수 테스트
['ap', 'ba', 'pi', 'gr']


In [146]:
# . 제거 (나중엔 필요 없음)
# term_frequency = list(map(lambda x: x[:-1], term_frequency))

# join list element
term_string = ','.join(term_frequency)
term_list = term_string.split(',')

# to lower 
term_list = list(map(lambda x: x.lower(), term_list))

In [147]:
# python basic count package
from collections import Counter

Counter(term_list).most_common()

[('oral health', 10),
 ('bone regeneration', 6),
 ('inflammation', 4),
 ('dentistry', 3),
 ('fibrosis', 3),
 ('saliva', 3),
 ('streptococcus mutans', 3),
 ('tooth extraction', 3),
 ('obesity', 3),
 ('orthodontic tooth movement', 3),
 ('osteoclast', 3),
 ('oral health-related quality of life', 3),
 ('rat', 3),
 ('dental implants', 3),
 ('peri-implantitis', 3),
 ('oral cancer', 2),
 ('oral surgery', 2),
 ('collagen membrane', 2),
 ('edentulism', 2),
 ('public health', 2),
 ('quality of life', 2),
 ('essential oils', 2),
 ('egfr', 2),
 ('stress', 2),
 ('cytokines', 2),
 ('biocompatibility', 2),
 ('autism spectrum disorder', 2),
 ('toothbrushing', 2),
 ('prevalence', 2),
 ('antimicrobial photodynamic therapy', 2),
 ('gbr', 2),
 ('histology', 2),
 ('cancer', 2),
 ('reactive oxygen species', 2),
 ('growth factors', 2),
 ('gene expression', 2),
 ('ridge preservation', 2),
 ('implants', 2),
 ('doxycycline', 2),
 ('membrane', 2),
 ('dentin', 2),
 ('apoptosis', 2),
 ('biomaterials', 2),
 ('prost